## Import stuff

In [4]:
import os
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import shutil
import random
from PIL import Image
import logging
import time
import sklearn.metrics as metrics


## Load training data from csv


In [5]:
val_data = pd.read_csv("val_distances.csv",index_col=0)
val_labels = pd.read_csv("val_labels.txt", header=None)


val_data.drop("anchor_left_distance_mn", axis=1, inplace=True)
val_data.drop("anchor_right_distance_mn", axis=1, inplace=True)
for extractor in ["xl","resnet"]:
    val_data[extractor] = val_data["anchor_left_distance_"+extractor]-val_data["anchor_right_distance_"+extractor]
    val_data.drop("anchor_left_distance_"+extractor, axis=1, inplace=True)
    val_data.drop("anchor_right_distance_"+extractor, axis=1, inplace=True)


X_val = np.array(val_data.iloc[:, :].values,  dtype=float)
Y_val = np.array(val_labels.iloc[:, :].values,  dtype=float).reshape(-1)

print("X shape: ", X_val.shape)
print("Y shape: ", Y_val.shape)



X shape:  (7476, 2)
Y shape:  (7476,)


In [6]:
pred = []
for row in X_val:
    score_left = row[0]+row[1]
    pred.append(0 if score_left>0 else 1)

pred = np.array(pred)
accuracy = metrics.accuracy_score(Y_val, pred<0.5)
accuracy

0.7683253076511504